In [1]:
import json
import numpy as np
from activation_functions import Tanh, Sigmoid
from mse import mse, mse_derivative
from mlp import Dense, train, predict
from utils import get_config_params, get_data, get_training_amount
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
with open('./config.json', 'r') as f:
    config = json.load(f)

ej, learning_rate, training_percentage, max_epochs, bias, beta, epsilon, optimizer = get_config_params(config)

input_data, expected_data = get_data(2)
training_amount = get_training_amount(len(input_data), training_percentage)

flattened_input = []
for i in range(len(input_data)):
    flattened_input.append([item for sublist in input_data[i] for item in sublist])

training_set = flattened_input[:training_amount]
training_expected = expected_data[:training_amount]
testing_set =  np.reshape(flattened_input[training_amount:], (len(input_data)-training_amount, 35, 1))
testing_expected = expected_data[training_amount:]
X = np.reshape(training_set, (training_amount, 35, 1))
Y = np.reshape(training_expected, (training_amount, 1, 1))
# print(X)
print(Y)

[[[0]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]]


In [3]:
network = [
        Dense(35, 32, optimizer_type=optimizer),
        Sigmoid(),
        Dense(32, 2,optimizer_type=optimizer),
        Sigmoid()
    ]
network


In [4]:
# train
train(network, mse, mse_derivative, X, Y, epochs=10000)

1/10000, error=0.4398944468877958
2/10000, error=0.4319931241604129
3/10000, error=0.4259891061459761
4/10000, error=0.4201785897289083
5/10000, error=0.41440061820561913
6/10000, error=0.40860631999342106
7/10000, error=0.40277525586741836
8/10000, error=0.39689779068599496
9/10000, error=0.3909696092279697
10/10000, error=0.3849896135165682
11/10000, error=0.37895886704540077
12/10000, error=0.37287990768408474
13/10000, error=0.36675641797287784
14/10000, error=0.3605932606611054
15/10000, error=0.3543962864609396
16/10000, error=0.3481717619135967
17/10000, error=0.34192567177604727
18/10000, error=0.33566302110128265
19/10000, error=0.32938725199228996
20/10000, error=0.3230998916784299
21/10000, error=0.31680051498782513
22/10000, error=0.31048705227459844
23/10000, error=0.30415641380430464
24/10000, error=0.2978053503708743
25/10000, error=0.291431443603265
26/10000, error=0.2850341199293512
27/10000, error=0.2786156025279659
28/10000, error=0.27218174672530315
29/10000, error=

In [5]:
points = []
for i in range(len(testing_set)):
    z = predict(network, testing_set[i])
    points.append([testing_set[i], testing_expected[i], z[0,0]])
for point in points:
    print(f"Input:" +
        #    {point[0]} +
             f"Expected:{point[1]} Result:{point[2]}")

Input:Expected:1 Result:9.954717662130257e-05
Input:Expected:0 Result:0.9887862699816052
Input:Expected:1 Result:0.011856677683114204
Input:Expected:0 Result:0.9823092619557015
Input:Expected:1 Result:0.00423346492534847
